In [1]:
import numpy as np 
import pandas as pd
import mygene
import matplotlib.pyplot as plt
import MiscFcns as MF
%matplotlib inline

In [2]:
#Load prediction dataset 
df = pd.read_csv('EC2_ICCT_subNet_de_agg_probs.csv')
GeneId = df['Gene']
df.head()

,Unnamed: 0,Gene,Y0h,Y0l,Y1h,Y1l,Y3h,Y3l
0,0,ENSG00000000003,0.240105,9.997225e-01,1.234691e-10,6.733560e-12,0.005367,2.949255e-07
1,1,ENSG00000000419,0.665309,2.850016e-01,1.229701e-08,2.012001e-15,0.000050,2.377142e-11
2,2,ENSG00000000457,0.665282,2.849000e-01,2.238065e-12,2.984028e-11,0.025684,7.106794e-05
3,3,ENSG00000000938,0.000953,7.429655e-07,7.256187e-08,3.230653e-06,0.122708,9.286597e-01
4,4,ENSG00000000971,0.016454,1.021360e-01,4.711436e-08,2.361356e-07,0.088079,7.580288e-01


In [3]:
#load GWAS dataset 
Gwas = pd.read_csv('Jensen_gene_summary.csv')
Gwas.head()

,Unnamed: 0,Mean,Min,Names
0,0,-0.416023,0.000484,SAMD11
1,1,-0.483331,0.000685,NOC2L
2,2,-0.520841,0.000049,KLHL17
3,3,-0.566113,0.000049,PLEKHN1
4,4,-0.521383,0.000049,PERM1


In [4]:
#load GWAS dataset 
Gwas2 = pd.read_csv('IGAP_gene_summary.csv')
Gwas2.head()

,Unnamed: 0,Mean,Min,Names
0,0,-0.380695,0.348300,SAMD11
1,1,-0.610730,0.133800,HES4
2,2,-0.492920,0.133800,ISG15
3,3,-0.648698,0.016870,AGRN
4,4,-0.911167,0.007672,RNF223


In [5]:
lgcl =((df['Y0l']) + (df['Y1l']) + (df['Y3l'])/3 > 0.5)
bla = np.argwhere(lgcl)
GenePred = list(GeneId[bla[:,0]])
GenePred2 = MF.ConvertToSymb(GenePred)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-4134...done.
Finished.
2 input query terms found no hit:
	[u'ENSG00000170647', u'ENSG00000267194']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


In [10]:
Y0 = (df['Y0l'])[bla[:,0]]
Y1 = (df['Y1l'])[bla[:,0]]
Y3 = (df['Y3l'])[bla[:,0]]
OR_0 = np.log10((Y0)/(1-Y0))
OR_1 = np.log10((Y1)/(1-Y1))
OR_3 = np.log10((Y3)/(1-Y3))

GW = np.zeros((len(Y0),))
GW_m = np.zeros((len(Y0),))
GW2_m = np.zeros((len(Y0),))

for i in range(len(GW)):
    
    In1 = np.argwhere(Gwas['Names']==GenePred2[i])
    In2 = np.argwhere(Gwas2['Names']==GenePred2[i])
        
    if len(In1)>0:
        GW[i] = Gwas['Mean'][In1[0]]
        GW_m[i] = Gwas['Min'][In1[0]]
        
    if len(In2)>0:
        GW2_m[i] = Gwas2['Min'][In2[0]]

        
Score = (1.0/6.0)*(OR_0 + OR_1 + OR_3) - 0.5*GW
        
d = {'Names':GenePred2,'DE':OR_0,'Mod-Net':OR_1,'G-Net':OR_3,'Score':Score,'Jensen':GW_m,'IGAP':GW2_m}
df2 = pd.DataFrame(data=d)
df2 = df2.sort_values(by='Score', ascending=False)



In [11]:
df2.head(n=25)


,DE,G-Net,IGAP,Jensen,Mod-Net,Names,Score
4923,-1.065414,2.614236,2.000000e-308,2.000000e-308,7.153369,APOC1,42.923028
4921,1.116040,1.101588,2.000000e-308,2.000000e-308,1.035915,APOE,41.746784
11626,3.621015,-7.720763,4.663000e-69,1.599058e-143,-1.846731,BCAM,5.881170
319,7.122674,11.764566,1.198000e-01,1.930000e-02,9.055416,CD74,4.918933
1716,8.008561,8.863077,1.072000e-03,2.954000e-15,8.696196,TREM2,4.652538
2463,-1.068735,-1.051141,2.799000e-21,7.065524e-50,4.989455,CLPTM1,4.577629
355,3.730522,8.641305,3.522000e-02,5.936116e-03,11.576028,DEF6,4.284926
7690,1.364253,13.005200,2.362000e-02,2.291650e-03,8.597759,SLC7A7,4.052869
5442,1.961811,6.815395,4.820000e-03,9.139000e-04,12.349285,DOCK2,3.715154
818,2.706846,6.488618,1.986000e-06,1.063896e-06,9.353182,SPI1,3.624854


In [9]:
df2.to_csv('./PaperFigs/DriverRanking_01212019.csv')

In [6]:
Y = pd.read_excel('./ResponseVec_040318.xlsx')
Y = Y.drop(['GeneID'], axis = 1)

In [20]:
bla2 = np.argwhere(Y.values==1)[:,0]
len(np.intersect1d(bla2,bla[:,0]))

25

In [12]:
Gwas[Gwas['Names']=='CD74']

,Unnamed: 0,Mean,Min,Names
5648,5648,-0.523648,0.0193,CD74
